## Data Loading
Loads in the big data matrix and the stfp matrix created by matlab

In [33]:
import scipy.io

def load_in_data():
    file = scipy.io.loadmat("data/simulationData.mat")
    data = file["data"]
    return data

def load_stfp():
    file = scipy.io.loadmat("data/stfp.mat")
    stfp = file["stfp"]
    return stfp


## Data processing
Processes the data just as in the matlab version

In [34]:
# Set the notebook directory as the working directory
%pwd
%cd C:\Users\Rafael\Documents\GitHub\BeyondHulten

C:\Users\Rafael\Documents\GitHub\BeyondHulten


In [36]:
import numpy as np
from scipy.io import loadmat

data = load_in_data()

grossy = np.reshape(data[:, 2], (46, 88)).T
capital = np.reshape(data[:, 3], (46, 88)).T
labor = np.reshape(data[:, 4], (46, 88)).T
vadd = labor + capital

removable_sectors = [59] + list(range(79, 88))
grossy = np.delete(grossy, removable_sectors, axis=0)
capital = np.delete(capital, removable_sectors, axis=0)
labor = np.delete(labor, removable_sectors, axis=0)
vadd = np.delete(vadd, removable_sectors, axis=0)

gross_sales = np.sum(grossy != 0, axis=1)
non_zero_rows = np.where(gross_sales)[0]
grossy = grossy[non_zero_rows, :]
capital = capital[non_zero_rows, :]
labor = labor[non_zero_rows, :]
vadd = vadd[non_zero_rows, :]

stfp = load_stfp()
Σ = np.cov(stfp, rowvar=False)
μ = np.mean(stfp, axis=1)

## Creating usable variables
Function that creates the metrics for a given year

In [38]:
def get_variables(year):
    IO = data[data[:, 0] == year, :]
    temp = [7, 59, 61] + list(range(79, 88))
    IO = np.delete(IO, temp, axis=1)
    IO = np.delete(IO, [0, 1, 2, 3, 4, 93], axis=1)
    IO = np.delete(IO, temp, axis=1)
    Ω = IO / np.sum(IO, axis=1, keepdims=True)
    α = vadd[:, year-1959] / grossy[:, year-1959]
    β = np.dot((np.eye(46) - np.diag(1 - α)).T @ Ω, grossy[:, year-1959])
    β[β < 0] = 0
    β = β / np.sum(β)
    λ = np.linalg.inv(np.eye(46) - np.diag(1 - α)) @ β
    L = λ * α

    return α, β, Ω, L, λ


In [59]:
year = 1973
IO = data[data[:, 0] == year, :]
temp = [7, 59, 61] + list(range(79, 88))
IO = np.delete(IO, temp, axis=1)
IO = np.delete(IO, [0, 1, 2, 3, 4], axis=1) #deleted 93 here
#IO = np.delete(IO, temp, axis=1) #why deleting it again?
#Ω = IO / np.sum(IO, axis=1, keepdims=True)
np.sum(IO, axis=1, keepdims=True) #Have zeero elements, can't divide through
#α = vadd[:, year-1959] / grossy[:, year-1959]
#β = np.dot((np.eye(46) - np.diag(1 - α)).T @ Ω, grossy[:, year-1959])
#β[β < 0] = 0
#β = β / np.sum(β)
#λ = np.linalg.inv(np.eye(46) - np.diag(1 - α)) @ β
#L = λ * α

array([[ 73265.058],
       [  1596.588],
       [  1374.003],
       [  2005.765],
       [  1790.021],
       [  1528.983],
       [ 11531.672],
       [     0.   ],
       [100963.432],
       [ 16962.221],
       [  7238.471],
       [ 11491.969],
       [ 49460.372],
       [ 30727.294],
       [ 32559.416],
       [  6051.804],
       [  2965.284],
       [  4623.179],
       [ 12225.807],
       [  4420.601],
       [  5792.324],
       [ 53813.784],
       [ 11460.877],
       [  2347.554],
       [  3568.559],
       [  4179.546],
       [  1840.47 ],
       [  3149.763],
       [  6568.919],
       [ 97442.274],
       [  2890.464],
       [ 20396.675],
       [ 17010.602],
       [  3376.722],
       [ 19289.785],
       [  7252.271],
       [  6595.53 ],
       [ 32279.448],
       [  3227.311],
       [ 27419.191],
       [ 13327.607],
       [  6181.649],
       [  1111.364],
       [ 19936.197],
       [  4558.526],
       [  6133.539],
       [  1426.   ],
       [ 1687

## Elasticities

In [ ]:
ε = .5;
θ = 0.001;
σ = .9;

## Objective Function

Here the objective function in `SimulationDerivs.m` is formulated, also the Jacobian is given, to helpt the solver.

In [39]:
def problem(X, A, β, Ω, α, ε, θ, σ, L):
    N = len(α)
    p = X[:N]
    y = X[N:]

    Out = np.zeros(2 * N, dtype=X.dtype)

    q = (Ω @ (p ** (1 - θ))) ** (1 / (1 - θ))
    w = p * (A ** ((ε - 1) / ε)) * (α ** (1 / ε)) * (y ** (1 / ε)) * L ** (-1 / ε)
    C = np.dot(w, L)

    Out[:N] = p - (A ** (ε - 1) * (α * w ** (1 - ε) + (1 - α) * q ** (1 - ε))) ** (1 / (1 - ε))
    Out[N:] = y - np.dot(np.dot(np.dot(np.dot(np.dot(y.T, np.diag(p) ** ε), np.diag(A) ** (ε - 1)), np.diag(q) ** (θ - ε)), np.diag(1 - α)), np.dot(np.dot(Ω, np.diag(p) ** (-θ)), β)) - np.dot(β.T, np.diag(p) ** (-σ) * C)

    return Out


In [44]:
def problem_jacobian(X, A, β, Ω, α, ε, θ, σ, L):
    N = len(α)
    p = X[:N]
    y = X[N:]

    Out = np.zeros((2 * N, 2 * N), dtype=X.dtype)

    q = (Ω @ (p ** (1 - θ))) ** (1 / (1 - θ))
    w = p * (A ** ((ε - 1) / ε)) * (α ** (1 / ε)) * (y ** (1 / ε)) * L ** (-1 / ε)
    C = np.dot(w, L)

    DQDP = (q ** θ * (p ** (-θ))[:, np.newaxis]) * Ω
    DWDP = np.diag(A ** ((ε - 1) / ε) * α ** (1 / ε) * y ** (1 / ε) * (1 / L) ** (1 / ε))
    DWDY = (1 / ε) * np.diag(p * A ** ((ε - 1) / ε) * α ** (1 / ε) * (y ** (1 - ε)) * L ** (-1 / ε))
    DCDP = np.dot(L.T, DWDP)
    DCDY = np.dot(L.T, DWDY)

    DOut1DP = np.eye(N) - np.diag(np.diag(A) ** (-1) * ((α * (w ** (1 - ε)) + (1 - α) * (q ** (1 - ε)))) ** (ε / (1 - ε))) @ (np.diag(α) @ np.diag(w ** (-ε)) @ DWDP + np.diag(1 - α) @ np.diag(q ** (-ε)) @ DQDP)

    DOut1DY = -np.diag(np.diag(A ** (-1)) * ((α * (w ** (1 - ε)) + (1 - α) * (q ** (1 - ε)))) ** (ε / (1 - ε))) @ (np.diag(α) @ np.diag(w ** (-ε)) @ DWDY)

    DOut2DP = -(ε * np.diag(p ** (-θ)) @ Ω.T @ np.diag((p ** (ε - 1)) * y * (q ** (θ - ε)) * (1 - α) * (A ** (ε - 1))) \
        + (θ - ε) * np.diag(p ** (-θ)) @ Ω.T @ np.diag((p ** ε) * y * (q ** (θ - ε - 1)) * (1 - α) * (A ** (ε - 1))) @ DQDP \
        - σ * np.diag(β * p ** (-σ - 1)) @ C + (β * (p ** (-σ))[:, np.newaxis] * DCDP.T)) \
        - θ * np.diag(p ** (-θ - 1)) @ np.diag(Ω.T @ np.diag((p ** ε) * (q ** (θ - ε)) * (1 - α) * (A ** (ε - 1))) @ y)

    DOut2DY = np.eye(N) - (np.diag(p) ** ε @ np.diag(A) ** (ε - 1) @ np.diag(q) ** (θ - ε) @ np.diag(1 - α) @ Ω @ np.diag(p ** (-θ))).T - (β * (p ** (-σ))[:, np.newaxis] * DCDY.T)
    
    OutDeriv = np.hstack((np.vstack((DOut1DP, DOut1DY)), np.vstack((DOut2DP, DOut2DY))))

    return OutDeriv


## Generating Random Shocks

In [45]:
from scipy.stats import multivariate_normal

def generate_random_shock(Σ, α, Ω, λ):
    n = len(Σ)
    cov_matrix = -0.5 * np.diag(Σ)
    A = np.exp(multivariate_normal.rvs(mean=np.zeros(n), cov=cov_matrix))
    
    inv_term = np.linalg.inv(np.eye(n) - np.diag(1 - α) @ Ω)
    log_A = np.log(A)
    init = np.concatenate((np.exp(-inv_term @ log_A), λ / np.exp(-inv_term @ log_A)))
    
    return A, init


## Solving for shocks

In [50]:
trials = 100
GDP = np.zeros(trials)
λ_sim = np.zeros((76, trials))
α, β, Ω, L, λ = get_variables(1973) 


IndexError: index 93 is out of bounds for axis 0 with size 82

In [46]:
import multiprocessing as mp
from scipy.optimize import NonlinearFunction, NonlinearProblem, root

# Parallel worker function
def parallel_worker(k):
    A, init = generate_random_shock(Σ, α, Ω, λ)
    p = [A, β, Ω, α, ε, θ, σ, L]
    prob = NonlinearProblem(lambda u: problem(u, *p), jac=lambda u: problem_jacobian(u, *p))
    sol = root(prob, init, method='hybr')

    x = np.real(sol.x)
    p = x[:76]
    q = x[76:]
    gdp = np.dot(p * (A ** ((ε - 1) / ε)) * (α ** (1 / ε)) * (q ** (1 / ε)) * (L ** (-1 / ε)), L)
    λ_sim[:, k] = (p * q) / gdp
    return gdp

# Parameters
trials = 100
GDP = np.zeros(trials)
λ_sim = np.zeros((76, trials))

α, β, Ω, L, λ = getVariables(1973)
Σ = np.array(...)  # Replace with appropriate values for Σ

# Parallel execution
num_processes = mp.cpu_count()
with mp.Pool(processes=num_processes) as pool:
    results = pool.map(parallel_worker, range(trials))
    GDP = np.array(results)

# Now you have both GDP and λ_sim populated after the parallel execution

ImportError: cannot import name 'NonlinearFunction' from 'scipy.optimize' (C:\Users\Rafael\Anaconda3\envs\Jupyter_plugin\lib\site-packages\scipy\optimize\__init__.py)

In [ ]:
trials = 100;
GDP = SharedVector(zeros(trials))
λ_sim = SharedMatrix(zeros(76, trials))
α, β, Ω, L, λ = getVariables(1973);
f = NonlinearFunction((u, p) -> problem(u, p...), jac=(u, p) -> problemJacobian(u, p...))

@distributed for k in 1:trials

    A, init = generateRandomShock(Σ, α, Ω, λ)

    p = [A, β, Ω, α, ε, θ, σ, L]

    ProbN = NonlinearProblem(f, init, p)
    sol = solve(ProbN, NLSolveJL(linesearch=HagerZhang(), method=:newton), reltol=1e-8, abstol=1e-8)

    x = real.(sol.u)

    p = @view x[1:76]
    q = @view x[77:end]
    GDP[k] = (p .* (A .^ ((ε - 1) / ε)) .* (α .^ (1 / ε)) .* (q .^ (1 / ε)) .* L .^ (-1 / ε))' * L
    λ_sim[:, k] .= (p .* q) ./ GDP[k]

end